In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

#### Montando entorno

In [15]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


In [3]:
import sys
sys.path.append(path_base + r"\models")

In [6]:
import joblib

In [33]:
# Importacion de sklearn para el modelo KNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [16]:
# Generando el Dataframe del dataset base
df = pd.read_csv( path_base + r"\data\procesed\blueTeam.csv")
test_data = pd.read_csv( path_base + r"\data\procesed\redTeam.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9879 entries, 0 to 9878
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                9879 non-null   int64  
 1   Wins                      9879 non-null   int64  
 2   WardsPlaced               9879 non-null   int64  
 3   WardsDestroyed            9879 non-null   int64  
 4   FirstBlood                9879 non-null   int64  
 5   Kills                     9879 non-null   int64  
 6   Deaths                    9879 non-null   int64  
 7   Assists                   9879 non-null   int64  
 8   EliteMonsters             9879 non-null   int64  
 9   Dragons                   9879 non-null   int64  
 10  Heralds                   9879 non-null   int64  
 11  TowersDestroyed           9879 non-null   int64  
 12  TotalGold                 9879 non-null   int64  
 13  AvgLevel                  9879 non-null   float64
 14  TotalExp

#### Definiendo Features

In [17]:
features = ["Kills", "Assists", "Deaths", "WardsPlaced", "WardsDestroyed", "GoldDiff", "ExperienceDiff"]

### Construyendo el modelo PCA
Este fragmento de código es esencial para preparar los datos antes de realizar cualquier análisis o modelado. La selección y estandarización de características son pasos cruciales para garantizar que los datos estén en un formato adecuado y que las características tengan una escala comparable, lo que puede mejorar la eficacia y la interpretación de los análisis posteriores.

In [25]:
# Elimina la columna de índice sin significado
data = df.drop("Unnamed: 0", axis=1)

#### División de Datos:
Divide tus datos en características (X) y la variable objetivo (y).

In [27]:
# Separa las características (X) y la variable objetivo (y)
X = data.drop("Wins", axis=1)
y = data["Wins"]

In [ ]:
# Divide tus datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Normalización de Datos:
Es importante normalizar tus datos antes de aplicar el PCA para asegurarte de que todas las características tengan la misma escala. Puedes usar la función StandardScaler de scikit-learn para hacer esto:

In [28]:
# Normaliza tus datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Entrenamiento del Modelo KNN:
A continuación, aplicaremos PCA a tus datos normalizados. Puedes especificar el número de componentes principales que deseas conservar. En este ejemplo, conservaremos todas las componentes

In [29]:
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

#### Visualización de la Precisión:


In [30]:
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = explained_variance_ratio.cumsum()

fig = px.line(x=range(1, len(cumulative_variance_ratio) + 1), y=cumulative_variance_ratio, title='Varianza Explicada Acumulativa')
fig.update_xaxes(title='Número de Componentes Principales')
fig.update_yaxes(title='Varianza Explicada Acumulativa')
fig.show()

#### Aplicando modelo con redTeam
El siguiente informe presenta la evaluación del modelo KNN aplicado a los datos del equipo redTeam. El objetivo de este análisis es determinar la capacidad del modelo entrenado en los datos del equipo blueTeam para generalizar y realizar predicciones precisas en el contexto de los datos del equipo redTeam.

In [32]:
# Supongamos que ya tienes un modelo de clasificación llamado "modelo" y un conjunto de prueba "X_test"
y_pred = pca.predict(test_data)

# Crea un DataFrame con las predicciones
result_df = pd.DataFrame({
    'Predicción': y_pred,
    'Real': test_data  # Supongo que y_test es la variable objetivo real de tus datos de prueba
})

# Cuenta la cantidad de aciertos y fallos
aciertos = result_df[result_df['Predicción'] == result_df['Real']]
fallos = result_df[result_df['Predicción'] != result_df['Real']]



AttributeError: 'PCA' object has no attribute 'predict'

In [36]:
# Crea un gráfico interactivo con Plotly
fig = go.Figure()

# Agrega un gráfico de barras para los aciertos
fig.add_trace(go.Bar(x=['Aciertos'], y=[len(aciertos)], name='Aciertos', marker_color='green'))

# Agrega un gráfico de barras para los fallos
fig.add_trace(go.Bar(x=['Fallos'], y=[len(fallos)], name='Fallos', marker_color='red'))

# Personaliza el diseño del gráfico
fig.update_layout(title='Resultados de la Predicción', xaxis_title='Resultados', yaxis_title='Cantidad')
fig.update_layout(barmode='stack')

# Muestra el gráfico interactivo
fig.show()

In [39]:

# Calcular la cantidad de filas en las que "Wins" y "KNN_Predictions" coinciden
coincidencias = test_data[(test_data["Wins"] == 1) & (test_data["KNN_Predictions"] == 1)]

# Calcular la cantidad de filas en las que "Wins" y "KNN_Predictions" no coinciden
no_coincidencias = test_data[(test_data["Wins"] == 0) & (test_data["KNN_Predictions"] == 1)]

# Crear un DataFrame con los totales
resultados = pd.DataFrame({"Resultado": ["Coincidencias", "No Coincidencias"],
                           "Cantidad": [len(coincidencias), len(no_coincidencias)]})

# Crear un gráfico de barras
fig = px.bar(resultados, x="Resultado", y="Cantidad",
             title="Eficiencia del Modelo (Coincidencias vs No Coincidencias)",
             labels={"Resultado": "Resultado", "Cantidad": "Cantidad"})

# Mostrar el gráfico
fig.show()

In [41]:
joblib.dump(knn, 'modelo_knn.pkl')


['modelo_knn.pkl']

In [44]:
notificar("Modelo KNN entrenado", "el modelo fue entrenado y exportado para su uso 'modelo_knn.pkl'",20)

NameError: name 'notificar' is not defined